# Linear Programming

Linear programming is a mathematical technique to achieve the best outcome (such as maximum profit, or least possible cost), whilst having various constraints.

For our course, we will be using the `pulp` library which you can find more information __[here](https://www.coin-or.org/PuLP/pulp.html)__.

## Problem 5

A trading company is looking for a way to maximize profit per transportation of their goods. 

The company has a train available with 3 wagons. When stocking the wagons they can choose between 4 types of cargo, each with its own specifications. How much of each
cargo type should be loaded on which wagon in order to maximize profit?

<br/>

Wagons Specification

| Train Wagon | Weight Capacity (Tonnes) | Space Capacity ($m^2$) |
| --- | --- | --- |
| w1 | 10 | 5000 |
| w2 | 8 | 4000 |
| 23 | 12 | 8000 |

<br/>

Cargo Specifications

| Cargo Type | Available (Tonne) | Volume ($m^2$) | Profit (per Tonne) |
| --- | --- | --- | --- |
| c1 | 18 | 400 | 2000 |
| c2 | 10 | 300 | 2500 |
| c3 | 5 | 200 | 5000 |
| c4 | 20 | 500 | 3500 |

<br/>

## Setting up the Problem

The first task is to set up the problem by:
- Creating an `LpProblem`
- Defining the variables (including any bounds, and data type)

In this case we have 12 variables that are obtained from the possible combinations of wagons and the type of cargo they can carry ($3wagons * 4cargotypes$). Their data type must be `Integer` since we cannot have partials.

In [28]:
from pulp import *

In [29]:
lp = LpProblem("Wagons", LpMaximize)

a = LpVariable(name="w1c1", lowBound=0, cat="Integer")
b = LpVariable(name="w1c2", lowBound=0, cat="Integer")
c = LpVariable(name="w1c3", lowBound=0, cat="Integer")
d = LpVariable(name="w1c4", lowBound=0, cat="Integer")
e = LpVariable(name="w2c1", lowBound=0, cat="Integer")
f = LpVariable(name="w2c2", lowBound=0, cat="Integer")
g = LpVariable(name="w2c3", lowBound=0, cat="Integer")
h = LpVariable(name="w2c4", lowBound=0, cat="Integer")
i = LpVariable(name="w3c1", lowBound=0, cat="Integer")
j = LpVariable(name="w3c2", lowBound=0, cat="Integer")
k = LpVariable(name="w3c3", lowBound=0, cat="Integer")
l = LpVariable(name="w3c4", lowBound=0, cat="Integer")

## Determine the Objective

Next step is to determine the objective

In this case we have to maximise profit so, we need to take care of that by multiplying each wagon with each possible type of cargo's profit.

So we have $max(p) = 2000a + 2500b + 5000c + 3500d + 2000e + 2500f + 5000g + 3500h + 2000i + 2500j + 5000k + 3500l $

This can be coded as following

In [30]:
lp.objective  = 2000*a + 2500*b + 5000*c + 3500*d + 2000*e + 2500*f + 5000*g + 3500*h + 2000*i + 2500*j + 5000*k + 3500*l

## Setting up the Constraints

The next step is to plot out the constraints. In this case we have the following:
- Wagon 1 weight cannot exceed 10 tonnes: $ a + b + c +d <= 10 $
- Wagon 2 weight cannot exceed 8 tonnes: $ e + f + g + h <= 8 $
- Wagon 3 weight cannot exceed 12 tonnes: $ i + j + k + l <= 12 $
- Wagon 1 space cannot exceed 5000 $m^2$ :  $400a + 300b + 200c + 500d <= 5000 $ 
- Wagon 2 space cannot exceed 4000 $m^2$ :  $400a + 300b + 200c + 500d <= 4000 $ 
- Wagon 3 space cannot exceed 8000 $m^2$ :  $400a + 300b + 200c + 500d <= 8000 $ 
- There are 18 tonnes available of Cargo 1 : $ a + e + i <= 18$
- There are 10 tonnes available of Cargo 2 : $ b + f + j <= 10$
- There are 5 tonnes available of Cargo 3 : $ c + g + k <= 5$
- There are 20 tonnes available of Cargo 4 : $ d + h + l <= 20$ 



In [31]:
lp.addConstraint(a+b+c+d <= 10, 'w1_limit')
lp.addConstraint(e+f+g+h <= 8, 'w2_limit')
lp.addConstraint(i+j+l <= 12, 'w3_limit')
lp.addConstraint(a+e+i <= 18, 'c1_limit')
lp.addConstraint(b+f+j <= 10, 'c2_limit')
lp.addConstraint(c+g+k <= 5, 'c3_limit')
lp.addConstraint(d+h+l <= 20, 'c4_limit')
lp.addConstraint(400*a + 300*b + 200*c + 500*d <= 5000, 'p1_limit')

## Solving the Problem

The last step is to solve the problem. This can be done by calling the `solve` function.

In [32]:
# solve the linear programming problem
# 1 = Optimal, 2 = Infeasible, 3 = Unbounded, 4 = Undefined, 5 = Not Solved

status = lp.solve(PULP_CBC_CMD(msg=0))
print(status)

# print solution
for var in lp.variables():
    print(var,'=',value(var))
    
print('OPT',value(lp.objective))

PulpSolverError: PULP_CBC_CMD: Not Available (check permissions on /usr/lib/python3/dist-packages/pulp/apis/../solverdir/cbc/linux/64/cbc)

If we look at the outcome we have:

status = 1 - The staus is optimal (it can be not solved, infeasible, unbounded or undef)

We should load:
- wagon 1 with 2 tonnes of cargo 2, and 8 tonnes of cargo 4
- wagon 2 with 2 tonnes of cargo 2, and 5 tonnes of cargo 3
- wagon 3 with 12 tonnes of cargo 4

The profit would be $107,500.
